In [1]:
import pandas as pd
from datetime import datetime
import plotly.express as px

In [2]:
gps_nucleo = pd.read_csv("C:/dashboard_onibus/dados/dados_gps.csv", sep=";")
faixas = pd.read_csv("C:/dashboard_onibus/dados/faixas_horarias.csv", sep=";")
linhas = pd.read_csv("C:/dashboard_onibus/dados/linhas_e_nucleos.csv", sep=";")
programacao = pd.read_csv("C://dashboard_onibus/dados/programacao_partidas.csv", sep=";")

In [3]:
del gps_nucleo['Linha Informada a SMTR']
del gps_nucleo['Linha Realizada pela SMTR']
del gps_nucleo['Veículo Consolidado pela SMTR']
del gps_nucleo ['Início da Viagem pela SMTR']
del gps_nucleo ['Término da Viagem pela SMTR']
del gps_nucleo ['Viagem Reconhecida']

In [4]:
gps_nucleo = pd.merge(gps_nucleo, linhas, on="Linha", how="left")

In [5]:
del gps_nucleo ['Tipo Linha']
del gps_nucleo ['Nome_Linha']

In [6]:
gps_nucleo['Início da Viagem'] = pd.to_datetime(gps_nucleo['Início da Viagem'], dayfirst=True)
gps_nucleo['Término da Viagem'] = pd.to_datetime(gps_nucleo['Término da Viagem'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

In [7]:
gps_nucleo['hora_viagem'] = gps_nucleo['Início da Viagem'].dt.time

In [8]:
gps_nucleo['_key'] = 1
faixas['_key'] = 1

In [9]:
merged = pd.merge(gps_nucleo, faixas, on='_key')

In [10]:
merged['hora_inicio'] = pd.to_datetime(merged['hora_inicio'], format='%H:%M:%S').dt.time
merged['hora_fim'] = pd.to_datetime(merged['hora_fim'], format='%H:%M:%S').dt.time

In [11]:
resultado = merged[(merged['hora_viagem'] >= merged['hora_inicio']) & (merged['hora_viagem'] <= merged['hora_fim'])]

In [12]:
resultado = resultado[['Início da Viagem', 'Faixa_Horaria']]

In [13]:
gps_nucleo = gps_nucleo.merge(resultado, on='Início da Viagem', how='left')

In [14]:
gps_nucleo = gps_nucleo.drop_duplicates()

In [15]:
partidas_executadas = gps_nucleo.groupby(['Linha', 'Faixa_Horaria']).size().reset_index(name='Total_Partidas')

In [16]:
partidas_executadas = partidas_executadas.sort_values(by=['Linha', 'Faixa_Horaria'])

In [17]:
programacao.columns = programacao.columns.str.strip()
partidas_executadas.columns = partidas_executadas.columns.str.strip()

In [18]:
partidas_executadas = partidas_executadas.rename(columns={'Total_Partidas': 'Executado'})

In [19]:
programacao = programacao.rename(columns={'Faixa': 'Faixa_Horaria', 'linha': 'Linha'})

In [20]:
comparativo = pd.merge(
    partidas_executadas,
    programacao.rename(columns={'Viagens Programadas': 'Programado'}),
    how='left',
    on=['Linha', 'Faixa_Horaria']
)


In [21]:
comparativo['Programado'] = comparativo['Programado'].fillna(0).astype(int)

In [22]:
comparativo['Programado'] = comparativo['Programado'].fillna(0).astype(int)

In [23]:
comparativo['% Execução'] = (comparativo['Executado'] / comparativo['Programado'].replace(0, 1)) * 100
comparativo['% Execução'] = comparativo['% Execução'].round(1)

In [24]:
comparativo.loc[comparativo['Programado'] == 0, '% Execução'] = 0

In [25]:
del comparativo ['Tipo Linha']

In [26]:
saida = "comparativo_programado_executado_com_nucleo.csv"

In [27]:
df_merged = pd.merge(
    comparativo,
    linhas[['Linha', 'Núcleo']],  # Mantém só a coluna necessária do núcleo
    on='Linha',
    how='left'  # Faz um left join para manter todas as linhas do comparativo
)

In [28]:
df_merged['% Execução'] = df_merged['% Execução'] / 100

In [29]:
df_merged['% Execução'] = df_merged['% Execução'].apply(lambda x: f"{x:.1%}")

In [30]:
df_merged

,Linha,Faixa_Horaria,Executado,Programado,Tipo Dia,Tipo de Programação,% Execução,Núcleo
0,108,03-06,12,0,NaN,NaN,0.0%,Gentileza
1,108,06-09,36,0,NaN,NaN,0.0%,Gentileza
2,108,09-12,32,0,NaN,NaN,0.0%,Gentileza
3,108,12-15,27,0,NaN,NaN,0.0%,Gentileza
4,110,03-06,15,0,NaN,NaN,0.0%,Gentileza
5,110,06-09,43,0,NaN,NaN,0.0%,Gentileza
6,110,09-12,34,0,NaN,NaN,0.0%,Gentileza
7,110,12-15,30,0,NaN,NaN,0.0%,Gentileza
8,112,03-06,15,0,NaN,NaN,0.0%,Gentileza
9,112,06-09,35,0,NaN,NaN,0.0%,Gentileza


In [31]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Linha                54 non-null     object
 1   Faixa_Horaria        54 non-null     object
 2   Executado            54 non-null     int64 
 3   Programado           54 non-null     int64 
 4   Tipo Dia             2 non-null      object
 5   Tipo de Programação  2 non-null      object
 6   % Execução           54 non-null     object
 7   Núcleo               54 non-null     object
dtypes: int64(2), object(6)
memory usage: 3.5+ KB


In [32]:
faixa_grafico = df_merged.loc[df_merged['Faixa_Horaria'] == '06-09']

In [33]:
faixa_grafico

,Linha,Faixa_Horaria,Executado,Programado,Tipo Dia,Tipo de Programação,% Execução,Núcleo
1,108,06-09,36,0,NaN,NaN,0.0%,Gentileza
5,110,06-09,43,0,NaN,NaN,0.0%,Gentileza
9,112,06-09,35,0,NaN,NaN,0.0%,Gentileza
13,163,06-09,21,0,NaN,NaN,0.0%,Gentileza
17,222,06-09,4,0,NaN,NaN,0.0%,Gentileza
21,309,06-09,62,0,NaN,NaN,0.0%,Central
25,315,06-09,30,0,NaN,NaN,0.0%,Central
30,460,06-09,82,0,NaN,NaN,0.0%,São Cristóvão
34,463,06-09,43,0,NaN,NaN,0.0%,São Cristóvão
38,538,06-09,15,0,NaN,NaN,0.0%,Central


In [34]:
faixa_grafico = faixa_grafico.sort_values(by='% Execução', ascending=False)

In [35]:
faixa_grafico [['Linha', '% Execução']]

,Linha,% Execução
1,108,0.0%
5,110,0.0%
9,112,0.0%
13,163,0.0%
17,222,0.0%
21,309,0.0%
25,315,0.0%
30,460,0.0%
34,463,0.0%
38,538,0.0%


In [36]:
# Criar coluna de texto formatada
faixa_grafico['text_percent'] = faixa_grafico['% Execução'].round(1).astype(str) + '%'

# Criar coluna de cor
faixa_grafico['color'] = faixa_grafico['% Execução'].apply(lambda x: 'green' if x >= 80 else 'red')

# Criar o gráfico
fig = px.bar(
    faixa_grafico,
    x='Linha',
    y='% Execução',
    text='text_percent',
    color='color',
    color_discrete_map={
        'green': 'green',
        'red': 'red'
    }
)

# Personalizar o eixo Y
fig.update_yaxes(
    dtick=10,
    range=[0, 110],
    title_text='% Execução'
)

# Melhorar aparência do texto nas barras
fig.update_traces(
    textposition='outside'
)

# Remover o fundo do gráfico
fig.update_layout(
    plot_bgcolor='white',      # Fundo da área do gráfico
    paper_bgcolor='white',     # Fundo total da imagem
    xaxis=dict(showgrid=False),  # Tirar linhas de grade no eixo X
    yaxis=dict(gridcolor='lightgrey')  # (Opcional) Manter grade Y em cinza claro
)

fig.show()




TypeError: '>=' not supported between instances of 'str' and 'int'